In [1]:
%load_ext autoreload
%autoreload 2
import os, pickle, csv, itertools, shutil, random, pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torchvision, torch
from torchvision import transforms
from omegaconf import OmegaConf as oc
from sklearn import datasets
# %matplotlib inline
# %config InlineBackend.figure_format='retina'

import utils.utils as utils
import evals.embed_evals as evals
import utils.plot_data as plot
import utils.gen_triplets as gen
def normalize(data):return (data-np.min(data)) / (np.max(data)-np.min(data))
import warnings
warnings.filterwarnings("ignore")
from scipy import stats
def get_ci(samples, confidence=0.95): return 2 * stats.sem(samples) * stats.t.ppf((1 + confidence) / 2., len(samples)-1)


In [ ]:
df = utils.get_wandb_runs("wv_squarelin")
eval_cols = ['test_triplet_acc','test_clf_acc', 'test_1nn_acc']
config_cols = ["seed", "weights", "filtered", "embed_dim", "lamda", "wandb_group", "wandb_project"]
df = utils.strip_df(df, eval_cols, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]

In [2]:
data = "wv_syn_bm"
out_dir = "link/" + data
pathlib.Path(out_dir+"/d=50").mkdir(parents=True, exist_ok=True)
pathlib.Path(out_dir+"/d=512").mkdir(parents=True, exist_ok=True)

eval_cols = ['NINO_ds_acc',  'NIFO_ds_acc', 'NI_h2h_d50', 'NO_h2h_d50', 'NI_h2h_d512', 'NO_h2h_d512', 'test_triplet_acc','test_clf_acc', 'test_1nn_acc', 'rNINO_ds_acc',]
config_cols = ["seed", "weights", "filtered", "embed_dim", "lamda", "wandb_group", "wandb_project"]

## aligns

In [3]:
df = utils.get_wandb_runs(data + "_aligns")
df = utils.strip_df(df, eval_cols, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]

In [4]:
filtered_df = df[df["filtered"]==True]
unfiltered_df = df[df["filtered"]==False]
filtered_50 = filtered_df[filtered_df["embed_dim"]==50]
filtered_512 = filtered_df[filtered_df["embed_dim"]==512]
unfiltered_50 = unfiltered_df[unfiltered_df["embed_dim"]==50]
unfiltered_512 = unfiltered_df[unfiltered_df["embed_dim"]==512]
print(len(filtered_50))
print(len(filtered_512))
print(len(unfiltered_50))
print(len(unfiltered_512))

60
60
0
0


In [30]:
filtered_50.to_csv(out_dir + "/d=50/" + data + "_filtered_d50.csv",index=False)
filtered_512.to_csv(out_dir + "/d=512/" + data + "_filtered_d512.csv",index=False)
# unfiltered_50.to_csv(out_dir + "/d=50/" + data + "_unfiltered_d50.csv",index=False)
# unfiltered_512.to_csv(out_dir + "/d=512/" + data + "_unfiltered_d512.csv",index=False)

In [6]:
filtered_512.groupby(["name","align"]).mean()

seed  filtered  embed_dim  lamda  NINO_ds_acc  NIFO_ds_acc  \
name   align                                                               
MTL0.2 0.69    1.0       1.0      512.0    0.2     0.622500     0.956667   
       0.84    1.0       1.0      512.0    0.2     0.596667     0.951667   
       0.89    1.0       1.0      512.0    0.2     0.810000     0.955833   
       0.95    1.0       1.0      512.0    0.2     0.649167     0.956667   
       0.97    1.0       1.0      512.0    0.2     0.856667     0.962500   
MTL0.5 0.69    1.0       1.0      512.0    0.5     0.575000     0.955000   
       0.84    1.0       1.0      512.0    0.5     0.604167     0.951667   
       0.89    1.0       1.0      512.0    0.5     0.798333     0.964167   
       0.95    1.0       1.0      512.0    0.5     0.652500     0.962500   
       0.97    1.0       1.0      512.0    0.5     0.877500     0.963333   
MTL0.8 0.69    1.0       1.0      512.0    0.8     0.584167     0.959167   
       0.84    1.0       1.0      512.0    0.8     0.608333     0.945833   
       0.89    1.0       1.0      512.0    0.8     0.845833     0.955000   
       0.95    1.0       1.0      512.0    0.8     0.639167     0.963333   
       0.97    1.0       1.0      512.0    0.8     0.885000     0.964167   
TN     0.69    1.0       1.0      512.0    0.0     0.624167     0.499167   
       0.84    1.0       1.0      512.0    0.0     0.596667     0.640000   
       0.89    1.0       1.0      512.0    0.0     0.728333     0.425833   
       0.95    1.0       1.0      512.0    0.0     0.671667     0.493333   
       0.97    1.0       1.0      512.0    0.0     0.858333     0.541667   

              NI_h2h_d50  NO_h2h_d50  NI_h2h_d512  NO_h2h_d512  \
name   align                                                     
MTL0.2 0.69     0.724306    0.810833     0.683056     0.780278   
       0.84     0.707083    0.830139     0.656111     0.783194   
       0.89     0.842639    0.914028     0.801111     0.927500   
       0.95     0.707917    0.834444     0.678333     0.796111   
       0.97     0.821806    0.883750     0.756528     0.920972   
MTL0.5 0.69     0.723333    0.824028     0.673611     0.796250   
       0.84     0.703056    0.822639     0.663472     0.803611   
       0.89     0.843472    0.904028     0.805556     0.932778   
       0.95     0.715556    0.825417     0.677500     0.815417   
       0.97     0.816389    0.846667     0.761111     0.900139   
MTL0.8 0.69     0.722222    0.835556     0.681250     0.812639   
       0.84     0.705694    0.789167     0.661944     0.756250   
       0.89     0.825694    0.857917     0.771111     0.905000   
       0.95     0.718056    0.822083     0.678611     0.831944   
       0.97     0.818750    0.857917     0.766250     0.907500   
TN     0.69     0.682222    0.795833     0.645417     0.787500   
       0.84     0.703611    0.823750     0.671250     0.782500   
       0.89     0.757778    0.963056     0.693333     0.969028   
       0.95     0.638333    0.851111     0.597500     0.838889   
       0.97     0.638472    0.931111     0.563750     0.958472   

              test_triplet_acc  test_clf_acc  test_1nn_acc  rNINO_ds_acc  
name   align                                                              
MTL0.2 0.69           0.832560      0.966667      0.967500      0.620833  
       0.84           0.843466      0.969167      0.970000      0.597500  
       0.89           0.977971      0.955833      0.961667      0.807500  
       0.95           0.869999      0.958333      0.961667      0.647500  
       0.97           0.966497      0.962500      0.965000      0.852500  
MTL0.5 0.69           0.828363      0.965000      0.966667      0.575000  
       0.84           0.842923      0.969167      0.969167      0.605000  
       0.89           0.974144      0.964167      0.969167      0.798333  
       0.95           0.872243      0.966667      0.965833      0.654167  
       0.97           0.963932      0.963333      0.965833     

## noisy

In [6]:
df = utils.get_wandb_runs(data + "_noisy")
df = utils.strip_df(df, eval_cols, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]
df["noise"] = [utils.find_float(x)[1] for x in df["wandb_group"]]

In [7]:
filtered_df = df[df["filtered"]==True]
unfiltered_df = df[df["filtered"]==False]
filtered_50 = filtered_df[filtered_df["embed_dim"]==50]
filtered_512 = filtered_df[filtered_df["embed_dim"]==512]
unfiltered_50 = unfiltered_df[unfiltered_df["embed_dim"]==50]
unfiltered_512 = unfiltered_df[unfiltered_df["embed_dim"]==512]
print(len(filtered_50))
print(len(filtered_512))
print(len(unfiltered_50))
print(len(unfiltered_512))

96
96
95
96


In [8]:
filtered_50.to_csv(out_dir + "/d=50/" + data + "_noisy" + "_filtered_d50.csv",index=False)
filtered_512.to_csv(out_dir + "/d=512/" + data + "_noisy" + "_filtered_d512.csv",index=False)
unfiltered_50.to_csv(out_dir + "/d=50/" + data + "_noisy" + "_unfiltered_d50.csv",index=False)
unfiltered_512.to_csv(out_dir + "/d=512/" + data + "_noisy" + "_unfiltered_d512.csv",index=False)

## num

In [9]:
eval_cols_num = eval_cols.copy()
eval_cols_num.remove("test_clf_acc")

In [10]:
df = utils.get_wandb_runs(data + "_num")
df = utils.strip_df(df, eval_cols_num, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]
df["num"] = [utils.find_float(x)[1] for x in df["wandb_group"]]
df["test_clf_acc"] = [0]*len(df)

In [11]:
filtered_df = df[df["filtered"]==True]
unfiltered_df = df[df["filtered"]==False]
filtered_50 = filtered_df[filtered_df["embed_dim"]==50]
filtered_512 = filtered_df[filtered_df["embed_dim"]==512]
unfiltered_50 = unfiltered_df[unfiltered_df["embed_dim"]==50]
unfiltered_512 = unfiltered_df[unfiltered_df["embed_dim"]==512]
print(len(filtered_50))
print(len(filtered_512))
print(len(unfiltered_50))
print(len(unfiltered_512))

81
84
0
0


In [12]:
filtered_50.to_csv(out_dir + "/d=50/" + data + "_num" + "_filtered_d50.csv",index=False)
filtered_512.to_csv(out_dir + "/d=512/" + data + "_num" + "_filtered_d512.csv",index=False)
# unfiltered_50.to_csv(out_dir + "/d=50/" + data + "_num" + "_unfiltered_d50.csv",index=False)
# unfiltered_512.to_csv(out_dir + "/d=512/" + data + "_num" + "_unfiltered_d512.csv",index=False)

## RESN

In [17]:
df = pd.read_csv("evals/RESN_evals_d50.csv")
df["align"] = [utils.find_float(x)[0] for x in df["group"]]
df = df.rename({"group":"wandb_group"},axis=1)
df["name"] = ["RESN"]*len(df)

In [18]:
for col in config_cols+eval_cols:
    if col not in df.columns: df[col] = [None]*len(df)

In [19]:
df.to_csv(out_dir + "/d=50/" + "RESN.csv",index=False)

In [20]:
df = pd.read_csv("evals/RESN_evals_d512.csv")
df["align"] = [utils.find_float(x)[0] for x in df["group"]]
df = df.rename({"group":"wandb_group"},axis=1)
df["name"] = ["RESN"]*len(df)

In [21]:
for col in config_cols+eval_cols:
    if col not in df.columns: df[col] = [None]*len(df)

In [22]:
df.to_csv(out_dir + "/d=512/" + "RESN.csv",index=False)

In [23]:
df.groupby("align").mean()

,seed,NINO_ds_acc,rNINO_ds_acc,NIFO_ds_acc
align,,,,
0.69,1.0,0.702500,0.702500,0.718333
0.84,1.0,0.700833,0.700833,0.757500
0.89,1.0,0.900833,0.900000,0.956667
0.95,1.0,0.768333,0.768333,0.833333
0.97,1.0,0.909167,0.908333,0.961667


## scratch

In [122]:
df[df["align"]==1].groupby(["name"]).mean()

,seed,filtered,embed_dim,lamda,align,NINO_ds_acc,NIFO_ds_acc,NI_h2h,NO_h2h,test_triplet_acc,test_clf_acc,test_1nn_acc,rNINO_ds_acc
name,,,,,,,,,,,,,
MTL0.2,1.0,1.0,512.0,0.2,1.0,0.986600,0.986600,0.862786,0.911083,0.972380,0.986600,0.989112,0.986600
MTL0.5,1.0,1.0,512.0,0.5,1.0,0.989950,0.979899,0.864740,0.898241,0.975604,0.979899,0.987437,0.985762
MTL0.8,1.0,1.0,512.0,0.8,1.0,0.989950,0.988275,0.849386,0.922390,0.974276,0.988275,0.989950,0.990787
TN,1.0,1.0,512.0,0.0,1.0,0.989112,0.991625,0.883305,0.945422,0.971179,0.556114,0.991625,0.989112


In [134]:
df[df["align"]==1][["name"]+eval_cols].groupby(["name"]).agg(get_ci)

,NINO_ds_acc,NIFO_ds_acc,NI_h2h,NO_h2h,test_triplet_acc,test_clf_acc,test_1nn_acc,rNINO_ds_acc
name,,,,,,,,
MTL0.2,0.007207,0.025986,0.062288,0.080156,0.030150,0.025986,0.028828,0.031415
MTL0.5,0.000000,0.033027,0.067449,0.177430,0.016216,0.033027,0.024966,0.031415
MTL0.8,0.000000,0.014414,0.073302,0.111309,0.010419,0.014414,0.012483,0.007207
TN,0.031415,0.007207,0.042706,0.048093,0.025330,0.226881,0.007207,0.031415


In [133]:
df[(df["align"]==1) & (df["name"]=="MTL0.5")]

,name,model,seed,weights,filtered,embed_dim,lamda,align,wandb_project,NINO_ds_acc,NIFO_ds_acc,NI_h2h,NO_h2h,test_triplet_acc,test_clf_acc,test_1nn_acc,rNINO_ds_acc
1,MTL0.5,MTL,2,"[1, 1, 0, 0]",True,512,0.5,1.0,wv_3d_linear,0.98995,0.972362,0.850084,0.860972,0.976678,0.972362,0.987437,0.979900
9,MTL0.5,MTL,1,"[1, 1, 0, 0]",True,512,0.5,1.0,wv_3d_linear,0.98995,0.984925,0.867253,0.901591,0.971938,0.984925,0.992462,0.984925
22,MTL0.5,MTL,0,"[1, 1, 0, 0]",True,512,0.5,1.0,wv_3d_linear,0.98995,0.982412,0.876884,0.932161,0.978195,0.982412,0.982412,0.992462


In [43]:
cis = []
for align in ["0.5","0.8","0.83","0.92","0.925","1"]:
    data = df[df["group"]==f"align={align}"]["NIFO_ds_acc_k=1"]
    cis.append(np.around(get_ci(data),3))

In [112]:
df = pd.read_csv("link/wv_3d_square/wv_3d_square_num_512.csv")
MTL = df[df["wandb_name"]=="MTL0.5"]

In [113]:
# unfiltered_MTL = TN[["filtered" not in x for x in TN["wandb_group"]]]
MTL = MTL[["0.925" in x for x in MTL["wandb_group"]]]

In [118]:
cis = []
for align in [0.5,0.25,0.125,0.0625,0.03125,0.015625,0.0078125]:
    data = MTL[MTL["wandb_group"]==f"align=0.925_p={align}"]["h2h_512"]
    cis.append(np.around(get_ci(data)/2,3))

In [119]:
cis

[0.006, 0.02, 0.043, 0.016, 0.034, 0.006, 0.045]